In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

from car import Car
from tools_cv import *
from tools_kinematics import *

In [36]:
# Constants
A = get_sym('A')  # car length
B = get_sym('B')  # car width / track
C = get_sym('C')  # car height
l = get_sym('l')  # wheelbase
r = get_sym('r')  # wheel radius
R = get_sym('R')  # turning radius (Ackerman)

beta_FL = get_sym(r'\beta', 'FL')
beta_FR = get_sym(r'\beta', 'FR')

vel_FL = get_sym(r'\phi', 'FL', dot=1)
vel_FR = get_sym(r'\phi', 'FR', dot=1)
vel_RL = get_sym(r'\phi', 'RL', dot=1)
vel_RR = get_sym(r'\phi', 'RR', dot=1)

theta = get_sym(r'\theta')
theta_dot = get_sym(r'\theta', dot=1)

alias = {}

# display cos(theta) as c\theta, sin(theta) as s\theta
alias.update({sp.sin(theta): sp.symbols(r"s\theta"),
              sp.cos(theta): sp.symbols(r"c\theta"),
              sp.sin(beta_FL): sp.symbols(r"s\beta_{FL}"),
              sp.cos(beta_FL): sp.symbols(r"c\beta_{FL}"),
              sp.sin(beta_FR): sp.symbols(r"s\beta_{FR}"),
              sp.cos(beta_FL): sp.symbols(r"c\beta_{FR}")})

In [38]:
L_FL = L_FR = B / 2
L_RL = L_RR = sp.sqrt(B**2 / 4 + l**2)

alpha_FL = sp.pi/2
alpha_FR = -sp.pi/2
alpha_RL = sp.pi/2 + sp.atan(2*l/B)
alpha_RR = -sp.pi/2 - sp.atan(2*l/B)

beta_RL = -sp.atan(2*l/B)
beta_RR = sp.atan(2*l/B) + sp.pi

In [39]:
# rolling condition
M_rolling_FL = get_mat_rolling(alpha_FL, beta_FL, L_FL)
M_rolling_FR = get_mat_rolling(alpha_FR, beta_FR, L_FR)
M_rolling_RL = get_mat_rolling(alpha_RL, beta_RL, L_RL)
M_rolling_RR = get_mat_rolling(alpha_RR, beta_RR, L_RR)

M_rolling = sp.Matrix([M_rolling_FL, M_rolling_FR,
                       M_rolling_RL, M_rolling_RR])
eqnprint(get_sym('M', 'rolling'), alias, M_rolling)

# so sliding condition
M_no_sliding_FL = get_mat_no_sliding(alpha_FL, beta_FL, L_FL)
M_no_sliding_FR = get_mat_no_sliding(alpha_FR, beta_FR, L_FR)
M_no_sliding_RL = get_mat_no_sliding(alpha_RL, beta_RL, L_RL)
M_no_sliding_RR = get_mat_no_sliding(alpha_RR, beta_RR, L_RR)

M_no_sliding = sp.Matrix([M_no_sliding_FL, M_no_sliding_FR,
                          M_no_sliding_RL, M_no_sliding_RR])
eqnprint(get_sym('M', 'no-sliding'), alias, M_no_sliding)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [40]:
RI0 = rotation(theta, 'z')
R0I = RI0.T
eqnprint(get_sym('R', 'I', '0'), alias, R0I)

<IPython.core.display.Math object>

In [41]:
M_angle = sp.Matrix([M_rolling_RL, M_rolling_RR, M_no_sliding_RL])
eqnprint(get_sym('M', 'angle'), alias, M_angle)

M_r = sp.eye(2)*r
M_r = M_r.row_insert(len(M_r), sp.zeros(1, 2))
eqnprint(get_sym('M', 'r'), alias, M_r)

M_vel = sp.Matrix([vel_FL, vel_FR, vel_RL, vel_RR])

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Forward Kinematics

In [42]:
M_temp = M_angle.inv().multiply(M_r)
matprint(sp.simplify(M_temp))

M_temp = M_temp.multiply(sp.Matrix([vel_RL, vel_RR]))
matprint(sp.simplify(M_temp))

M_fwd = RI0.multiply(M_temp)
M_fwd = sp.simplify(M_fwd)
eqnprint(get_sym('M', 'forward'), alias, M_fwd)

Matrix([
[                                         r/2,                                         r/2],
[                                      -l*r/B,                                       l*r/B],
[-r*sqrt(1 + 4*l**2/B**2)/sqrt(B**2 + 4*l**2), r*sqrt(1 + 4*l**2/B**2)/sqrt(B**2 + 4*l**2)]])

Matrix([
[                                              r*(\dot{\phi_{RL}} + \dot{\phi_{RR}})/2],
[                                           l*r*(-\dot{\phi_{RL}} + \dot{\phi_{RR}})/B],
[r*sqrt((B**2 + 4*l**2)/B**2)*(-\dot{\phi_{RL}} + \dot{\phi_{RR}})/sqrt(B**2 + 4*l**2)]])

<IPython.core.display.Math object>

## Inverse Kinematics

In [43]:
M_temp_left = M_r.T.multiply(M_r)

M_temp_right = M_r.T.multiply(M_angle)
matprint(sp.simplify(M_temp_right))

M_temp = M_temp_left.inv().multiply(M_temp_right)
matprint(sp.simplify(M_temp))

M_inverse = M_temp.multiply(R0I)
M_inverse = sp.simplify(M_inverse)
eqnprint(get_sym('M', 'inverse'), alias, M_inverse)

Matrix([
[r, 0, -r*sqrt(B**2 + 4*l**2)/(2*sqrt(1 + 4*l**2/B**2))],
[r, 0,  r*sqrt(B**2 + 4*l**2)/(2*sqrt(1 + 4*l**2/B**2))]])

Matrix([
[1/r, 0, -sqrt(B**2 + 4*l**2)/(2*r*sqrt(1 + 4*l**2/B**2))],
[1/r, 0,  sqrt(B**2 + 4*l**2)/(2*r*sqrt(1 + 4*l**2/B**2))]])

<IPython.core.display.Math object>

From above, by knowing $\dot{^IP}$, we can know the values of $\dot{\phi_{RL}}$ and $\dot{\phi_{RR}}$.

We can then use them to solve $R$, $\beta_{FL}$, $\beta_{FL}$, $\dot{\phi_{FL}}$ and $\dot{\phi_{FR}}$.

In [46]:
# Turning radius
R_inv = B/2 * ((vel_RL + vel_RR) / (vel_RL - vel_RR))
eqnprint(R, alias, R_inv)

# front wheels beta
beta_FL_inv = -sp.atan(l / (R + B/2))
eqnprint(beta_FL, alias, beta_FL_inv)
beta_FR_inv = -sp.atan(l / (R - B/2))
eqnprint(beta_FR, alias, beta_FR_inv)

# front wheels speed
vel_FL_inv = theta_dot * sp.sqrt(l**2 + (R+B/2)**2) / r
eqnprint(vel_FL, alias, vel_FL_inv)
vel_FR_inv = theta_dot * sp.sqrt(l**2 + (R-B/2)**2) / r
eqnprint(vel_FR, alias, vel_FR_inv)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>